## Factory

Used when you want to separate **Object Creation** and **Object Usage**

- `Product` -> The class whose objects are created by the factory
- `ProductFactory` -> The class responsible for creating and returning the objects of the product
  - There might be some logic within this to select what Product to make

**How is it different from Builder?**
- In builder the object creation itself is done in steps
- Here, its done normally -> but what object to create has some logic that we want to keep in factory 

In [1]:
from abc import ABC, abstractmethod 

class DBConnection(ABC):
    @abstractmethod
    def connect(self, creds: dict):
        pass

class DBConnectionFactory(ABC):
    @abstractmethod
    def create_connection(self) -> DBConnection:
        pass

In [2]:
class MySQLConnection(DBConnection):
    def connect(self, creds: dict):
        print(f"Connecting to MySQL with {creds}")

class PostgresConnection(DBConnection):
    def connect(self, creds: dict):
        print(f"Connecting to MongoDB with {creds}")

class SQLConnectionFactory(DBConnectionFactory):
    def create_connection(self, db_name: str) -> DBConnection:
        if db_name == "mysql": return MySQLConnection()
        elif db_name == "postgres": return PostgresConnection()
        else: raise ValueError("Unknown database type")

# Example usage
factory = SQLConnectionFactory()
mysql_conn = factory.create_connection("mysql")
mysql_conn.connect({})

postgres_conn = factory.create_connection("postgres")
postgres_conn.connect({})

Connecting to MySQL with {}
Connecting to MongoDB with {}


## Abstract Factory

## Builder


**Used when the object creation is complex and is done in Multiple Steps**

**FLOW**
- `Product` -> the object that is being built ( complex )
- `Builder` -> the class that builds the product ( each method returns `self` to chain together). **This creates a empty product on init**
- `Director` -> Orchestrator that calls `builder` with relevant parts to build

In [41]:
from abc import ABC, abstractmethod
from typing import Self
from dataclasses import dataclass

class Car:
    def __init__(self):
        self.parts = []
    
    def __str__(self):
        return " | ".join(self.parts)
    
@dataclass
class CarConfig:
    engine: str
    chasis: str
    paint: str

class Builder(ABC):
    @abstractmethod
    def build_engine(self, engine: str) -> Self:
        pass

    @abstractmethod
    def build_chasis(self, chasis: str) -> Self:
        pass

    @abstractmethod
    def build_paint(self, paint: str) -> Self:
        pass
    
    @abstractmethod
    def get_final_car(self) -> Car: # Car here should be derived from Product class
        pass


In [ ]:
class CarBuilder(Builder):
    def __init__(self):
        self.car = Car() # INIT CAR OBJECT -> You can take this as INPUT to INIT
    
    def build_engine(self, engine: str) -> Self:
        self.car.parts.append("Engine: "+engine)
        return self

    def build_chasis(self, chasis: str) -> Self:
        self.car.parts.append("Chasis: "+chasis)
        return self

    def build_paint(self, paint: str) -> Self:
        self.car.parts.append("Paint: "+paint)
        return self

    def get_final_car(self) -> Car:
        return self.car
    

class Director:
    def __init__(self, builder: Builder):
        self.builder = builder # THIS CAN BE HARD CODED ALSO
    
    def build_car(self, car_config: CarConfig):
        return self.builder.build_chasis(car_config.chasis)\
            .build_engine(car_config.engine)\
            .build_paint(car_config.paint)\
            .get_final_car()

In [43]:
car_builder = CarBuilder()
car_director = Director(car_builder)

ready_car = car_director.build_car(CarConfig(engine="V8",chasis="Titanium",paint="Chrome Black"))
print(ready_car)

Chasis: Titanium | Engine: V8 | Paint: Chrome Black


## Singleton

1. Why override `__new__(cls)`?
   - In python when you create a object it calls the `__call__(self)` that internally calls 
    1. `__new__(cls)` -> this creates instance of object and passes it to `__init__(self)`
    2. `__init__(self)` -> this inits the instance ( `self` is actually created and pased by `__new__`)
   
2. How to pass arguments?
    - Just add `*args` and `**kwargs` to `__new__` -> it automatically passes them to `init`
    - Dont pass anything other than `cls` while called `super().__new__(cls)`

3. How to stop multiple inits?
   - Have another variable called `_initialised` to check inits
   - Initially its a class variable -> after first init it becomes instance variable

- Why you may never need this in Python? https://www.thepythoncodingstack.com/p/creating-a-singleton-class-in-python
  - Solution: Create new file and at bottom of that file create a new object of that class

In [4]:
from dataclasses import dataclass

@dataclass
class DBCredentials:
    key: str
    endpoint: str
    username: str

    def __str__(self):
        return f"{self.key} -> {self.endpoint} -> {self.username}"


In [9]:
class DatabaseConnection:
    _instance = None
    _initialised = None

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            print("Creating new instance")
            cls._instance = super().__new__(cls)
        
        return cls._instance

    def __init__(self, credentials: DBCredentials):
        if not self._initialised:
            print("Initing with DB credentials..", credentials)
            self.creds = credentials
            self._initialised = True

    def check_credentials(self):
        print("Checking creds..", self.creds)

In [10]:
creds = DBCredentials("xxxx","https://localhost:1111","ashish")

dbconn1 = DatabaseConnection(creds)
dbconn2 = DatabaseConnection(creds)

print()

print(dbconn1 is dbconn2)
print(id(dbconn1))
print(id(dbconn2))

print()

dbconn1.check_credentials()
dbconn2.check_credentials()

Creating new instance
Initing with DB credentials.. xxxx -> https://localhost:1111 -> ashish

True
138025018076736
138025018076736

Checking creds.. xxxx -> https://localhost:1111 -> ashish
Checking creds.. xxxx -> https://localhost:1111 -> ashish


## Prototype

**Used when creating objects is expensive -> you just need a copy of that object**
- Can be either shallow or deep Copy

Example: 
- You have a FileParser object which is time-taking to instantiate ( opens up a file -> does some processing -> holds the file pointers )
- Now, you need to use this same file in 3 different places
- Instead of making 3 diff objects, just clone it

**Note**: In python `copy.copy()` and `copy.deepcopy()` do the same thing without much effort

In [4]:
from typing import Self
from abc import ABC, abstractmethod
import copy

class Cloneable(ABC):
    @abstractmethod
    def clone(self) -> Self:
        pass

In [8]:
class FileParserReference(Cloneable):
    def __init__(self, file_path: str):
        print("Some heavy initialization with file path:", file_path)
        self.file_path = file_path

    def clone(self) -> Self:
        return copy.copy(self)

    def __str__(self):
        return f"FileParserReference(file_path={self.file_path})"

In [9]:
file1 = FileParserReference("/path/to/file1.txt")
file2 = file1.clone()

print(file1, id(file1))
print(file2, id(file2))

Some heavy initialization with file path: /path/to/file1.txt
FileParserReference(file_path=/path/to/file1.txt) 129752763263904
FileParserReference(file_path=/path/to/file1.txt) 129752763721808
